In [ ]:
    # def _get_obs(self):
    #     position = self.data.qpos.flat.copy()
    #     velocity = self.data.qvel.flat.copy()

    #     if self._exclude_current_positions_from_observation:
    #         position = position[1:]

    #     observation = np.concatenate((position, velocity)).ravel()
    #     return observation

In [ ]:
    # def _get_obs1(self):
        # Robot joint positions and velocities
        robot_qpos = self.data.qpos
        robot_qvel = self.data.qvel

        return robot_qpos, robot_qvel   
        # # Ball's position and velocity
        # ball_pos = self.data.get_body_xpos("ball")
        # ball_vel = self.data.get_body_xvelp("ball")
        
        # # Goal position - Assuming it's static, you can hardcode it or fetch it from XML if it's dynamic
        # goal_pos = np.array([goal_x, goal_y, goal_z])

        # return np.concatenate([robot_qpos, robot_qvel, ball_pos, ball_vel, goal_pos])

    # def print_obs(self):
    #     print("Observation len: \n", len(self._get_obs1()[0]), len(self._get_obs1()[1]))
    #     print("Observation qpos: \n", (self._get_obs1()[0]))
    #     print("Observation qvel: \n", (self._get_obs1()[1]))

    # def print_obs1(self):
    #     qpos = self.data.qpos
    #     qvel = self.data.qvel

    #     # Robot's positions and velocities based on its joint definitions
    #     robot_qpos = qpos[:13]
    #     robot_qvel = qvel[:18]

    #     # Ball's positions and velocities based on its "free" joint definition
    #     ball_qpos_start = 13  # Start index for ball's qpos (immediately after robot's qpos)
    #     ball_qpos = qpos[ball_qpos_start:ball_qpos_start+7]
    #     ball_qvel_start = 18  # Start index for ball's qvel (immediately after robot's qvel)
    #     ball_qvel = qvel[ball_qvel_start:ball_qvel_start+6]

    #     # Organize into a dictionary for clarity
    #     obs = {
    #         "robot_qpos": robot_qpos,
    #         "robot_qvel": robot_qvel,
    #         "ball_qpos": ball_qpos,
    #         "ball_qvel": ball_qvel,
    #     }

    #     # print obs
    #     print(obs)

In [ ]:
    def print_obs1(self):
        qpos = self.data.qpos
        qvel = self.data.qvel

        # Robot's positions and velocities based on its joint definitions
        robot_qpos = qpos[:13]
        robot_qvel = qvel[:18]

        # Ball's positions and velocities based on its "free" joint definition
        ball_qpos_start = 13
        ball_qpos = qpos[ball_qpos_start:ball_qpos_start+7]
        ball_qvel_start = 18
        ball_qvel = qvel[ball_qvel_start:ball_qvel_start+6]

        # Print formatted data
        print("-" * 40)
        print("Robot QPOS:")
        for i, value in enumerate(robot_qpos):
            print("Joint {0}: {1:.4f}".format(i+1, value))

        print("\nRobot QVEL:")
        for i, value in enumerate(robot_qvel):
            print("Joint {0}: {1:.4f}".format(i+1, value))

        print("\nBall QPOS (Position & Quaternion):")
        ball_pos_labels = ["X", "Y", "Z"]
        ball_quat_labels = ["W", "I", "J", "K"]
        for i, value in enumerate(ball_qpos[:3]):
            print("{0}-position: {1:.4f}".format(ball_pos_labels[i], value))
        for i, value in enumerate(ball_qpos[3:]):
            print("{0}-quaternion: {1:.4f}".format(ball_quat_labels[i], value))

        print("\nBall QVEL (Linear & Angular):")
        ball_lin_vel_labels = ["X", "Y", "Z"]
        ball_ang_vel_labels = ["Roll", "Pitch", "Yaw"]
        for i, value in enumerate(ball_qvel[:3]):
            print("{0}-velocity: {1:.4f}".format(ball_lin_vel_labels[i], value))
        for i, value in enumerate(ball_qvel[3:]):
            print("{0}-velocity: {1:.4f}".format(ball_ang_vel_labels[i], value))
        
        print("-" * 40)

In [ ]:
    # def reset_model0(self):
    #     noise_low = -self._reset_noise_scale
    #     noise_high = self._reset_noise_scale

    #     qpos = self.init_qpos + self.np_random.uniform(
    #         low=noise_low, high=noise_high, size=self.model.nq
    #     )
    #     qvel = (
    #         self.init_qvel
    #         + self._reset_noise_scale * self.np_random.standard_normal(self.model.nv)
    #     )

    #     self.set_state(qpos, qvel)

    #     observation = self._get_obs()
    #     return observation

In [ ]:
    def reset_model0(self):
        # Initial positions and velocities
        qpos = np.array(self.init_qpos)
        qvel = np.array(self.init_qvel)

        # Modify robot's orientation
        robot_root_orientation_indices = slice(10, 14)

        theta = math.radians(90)
        qw = math.cos(theta / 2)
        qx = 0
        qy = 0
        qz = math.sin(theta / 2)

        robot_orientation = [qw, qx, qy, qz]
        qpos[robot_root_orientation_indices] = robot_orientation  # Assuming you know the indices

        # Let's say you want to introduce some randomness in the initial position of the ball
        ball_offset = np.random.uniform(-0.05, 0.05, size=(3,))
        qpos[:3] += ball_offset  # Assuming ball's qpos is first

        # Set the state
        self.set_state(qpos, qvel)

        return self._get_obs()

In [ ]:
    # def reset0(self):
    #     # Reset simulation state
    #     self.reset_model()

    #     # Set initial robot state (for example purposes, setting to zeros)
    #     # initial_qpos = np.zeros(self.model.nq)
    #     # initial_qvel = np.zeros(self.model.nv)
    #     # self.set_state(initial_qpos, initial_qvel)

    #     # Optionally reset positions of other objects (like ball or goal)
    #     # For example:
    #     # ball_id = self.model.body_name2id("ball")
    #     # self.model.body_pos[ball_id] = [random_x, random_y, random_z]

    #     # Reset episode-specific variables, if any
    #     # Example: 
    #     # self.steps_taken = 0

    #     # Return the initial observation
    #     return self._get_obs()

    def reset(self):
        # self.sim.reset()  # Reset the simulator
        self.reset_model()  # Reset positions, velocities, etc.
        
        # Optional: Introduce slight randomness in starting conditions
        self.data.qpos[self.robot_joint_position_indices] += np.random.randn(self.robot_joint_count) * 0.01
        self.data.qvel[self.robot_joint_velocity_indices] += np.random.randn(self.robot_joint_count) * 0.01
        self.data.qpos[self.ball_position_indices] += np.random.randn(3) * 0.01
        self.data.qvel[self.ball_velocity_indices] += np.random.randn(3) * 0.01
        
        # Recompute the simulation after any changes
        # self.sim.forward()
        
        # Return the initial observation
        return self._get_obs()

In [ ]:
        # self.observation_space = Box(
        #     low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64)

In [ ]:
#last old and working _get_obs but array
    def _get_obs(self):
        qpos = self.data.qpos
        qvel = self.data.qvel

        # Flatten the observations into an array
        obs = np.concatenate([
            qpos[:7],      # ball qpos
            qvel[:6],      # ball qvel
            qpos[7:14],    # robot root qpos
            qvel[6:12],    # robot root qvel
            qpos[14:26],   # robot joints qpos
            qvel[12:24]    # robot joints qvel
        ])

        return obs

In [ ]:
#dict def of obs space        
# self.max_joint_angle = 4.18879
        # #observation space
        # self.observation_space = spaces.Dict({
        # "ball_position": spaces.Box(low=-np.inf, high=np.inf, shape=(3,)),
        # "goal_position": spaces.Box(low=-np.inf, high=np.inf, shape=(3,)),  # Changed to 3D since it should be consistent with ball_position
        # "ball_trajectory": spaces.Box(low=-np.inf, high=np.inf, shape=(15, 3)),
        # "robot_position": spaces.Box(low=-np.inf, high=np.inf, shape=(3,)),
        # "robot_orientation": spaces.Box(low=-np.inf, high=np.inf, shape=(4,)),  # Quaternions for 3D orientation
        # "robot_joint_positions": spaces.Box(low=-self.max_joint_angle, high=self.max_joint_angle, shape=(self.robot_joint_count,)),
        # "robot_joint_velocities": spaces.Box(low=-np.inf, high=np.inf, shape=(self.robot_joint_count,)),
        # "motion_indicator": spaces.Discrete(2)})  # 0 for not moving, 1 for moving

In [ ]:
#old defs

exclude_current_positions_from_observation = 7  # 3 for position and 4 for orientation
obs_size = (
    3  # ball_position
    + 2  # goal_position
    + 45 #or 30  # ball_trajectory, depending on whether it's 2D or 3D
    + 3  # robot_position
    + 4  # robot_orientation
    + self.data.qpos.size - exclude_current_positions_from_observation  # exclude robot's root position and orientation
    + self.data.qvel.size)  # robot joint velocities

methods needed:
-step
-reward
-action_space

methods done:
-reset_model
-_get_obs


In [ ]:
    def _get_obs1(self):
        qpos = self.data.qpos
        qvel = self.data.qvel

        # Standard observations from MuJoCo
        # saved for later for planning policy
        standard_obs = np.concatenate([
            qpos[:7],      # ball qpos
            qvel[:6],      # ball qvel

            qpos[7:14],    # robot root qpos
            qvel[6:12],    # robot root qvel

            qpos[14:26],   # robot joints qpos
            qvel[12:24]    # robot joints qvel
        ])

        # Combine joint positions and orientations history into one variable
        combined_robot_state_history = np.concatenate([self.robot_joint_positions_history,
                                                        self.robot_orientation_history], axis=1).flatten()

        #Control policy state/observation space
        control_policy_observation = np.concatenate([
            np.array([self.motion_phase_selector]),                # motion selector
                        self.bezier_parameters.flatten(),          # Bézier parameters
                        np.array([self.motion_phase_time_span,     # duration of current motion phase
                        self.motion_phase_progress]),              # current motion phase progress
                        combined_robot_state_history,              # combined history of joint positions and orientations
                        self.action_history.flatten()])            # last 6 actions history

        return control_policy_observation

In [ ]:
# # Data storage lists
# # 26 lists to store data for each joint
# data_store = [[] for _ in range(26)]

# # Simulate and collect data for 60 seconds
# start_time = time.time()
# max_runtime = 60

# env.reset()
# time.sleep(1)

# while time.time() - start_time < max_runtime:
#     x = env.action_space.sample()
#     # obs, reward, done, info = 
#     env.step(x)  

#     # Store qpos data
#     for i in range(26):
#         data_store[i].append(env.data.qpos[i])

#     time.sleep(0.1)
#     env.render()

# # Create separate plots
# time_axis = np.arange(len(data_store[0]))  

# for i in range(26):
#     plt.figure()  # Create a new figure for each plot
#     plt.plot(time_axis, data_store[i])
#     plt.xlabel("Time Steps")
#     plt.ylabel(f"qpos Element {i}")  # Generic label
#     plt.title(f"Change in qpos Element {i}")
#     plt.show()

In [ ]:
# class SimState:
#     """
#         Takes in env.data.qpos and env.data.qvel and processes
#         them with labels into:
#             -trunk_qpos_dict
#             -joints_qpos_dict
#             -ball_qpos_dict

#             -trunk_qvel_dict
#             -joints_qvel_dict
#             -ball_qvel_dict
#     """
    
#     def __init__(self, env_data):
#         self.qpos = env_data.qpos
#         self.qvel = env_data.qvel
#         self.sensordata = env_data.sensordata
#         self.sensor = env_data.sensor

#         self.joints = ["FR_hip", "FR_thigh", "FR_calf",
#                         "FL_hip", "FL_thigh", "FL_calf",
#                         "RR_hip", "RR_thigh", "RR_calf",
#                         "RL_hip", "RL_thigh", "RL_calf"]
        
#         ###############################################################
#         #from qpos and qvel
#         self.ball_qpos = self.qpos[:7]
#         self.trunk_qpos = self.qpos[7:14]
#         self.joints_qpos = self.qpos[14:]
        
#         self.ball_qvel = self.qvel[:6]
#         self.trunk_qvel = self.qvel[6:12]
#         self.joints_qvel = self.qvel[12:]

#         self.trunk_qpos_dict = self.create_trunk_qpos_dict()
#         self.joints_qpos_dict = self.create_joints_qpos_dict()
#         self.ball_qpos_dict = self.create_ball_qpos_dict()

#         self.trunk_qvel_dict = self.create_trunk_qvel_dict()
#         self.joints_qvel_dict = self.create_joints_qvel_dict()
#         self.ball_qvel_dict = self.create_ball_qvel_dict()

#         ###############################################################
#         #from sensor data
#         self.sensed_joints_qpos = self.sensordata[:12]
#         self.sensed_joints_qvel = self.sensordata[12: 24]

#         self.sensed_joints_qpos_dict = self.create_sensed_joints_qpos_dict()
#         self.sensed_joints_qvel_dict = self.create_sensed_joints_qvel_dict()

#     def create_trunk_qpos_dict(self):
#         trunk_qpos_dict = {
#             'trunk_pos_x': self.trunk_qpos[0],
#             'trunk_pos_y': self.trunk_qpos[1],
#             'trunk_pos_z': self.trunk_qpos[2],
#             'trunk_orient_qw': self.trunk_qpos[3],
#             'trunk_orient_qx': self.trunk_qpos[4],
#             'trunk_orient_qy': self.trunk_qpos[5],
#             'trunk_orient_qz': self.trunk_qpos[6]}
        
#         return trunk_qpos_dict

#     def create_joints_qpos_dict(self):
#         leg_names = ['FR', 'FL', 'RR', 'RL']
#         joint_names = ['hip_pos', 'thigh_pos', 'calf_pos']
#         joints_qpos_dict = {}
#         for i, leg in enumerate(leg_names):
#             for j, joint in enumerate(joint_names):
#                 key = f"{leg}_{joint}"
#                 joints_qpos_dict[key] = self.joints_qpos[i * 3 + j]
#         return joints_qpos_dict

#     def create_ball_qpos_dict(self):
#         ball_qpos_dict =  {
#             'ball_pos_x': self.ball_qpos[0],
#             'ball_pos_y': self.ball_qpos[1],
#             'ball_pos_z': self.ball_qpos[2],
#             'ball_orient_qw': self.ball_qpos[3],
#             'ball_orient_qx': self.ball_qpos[4],
#             'ball_orient_qy': self.ball_qpos[5],
#             'ball_orient_qz': self.ball_qpos[6]}
        
#         return ball_qpos_dict

#     def create_trunk_qvel_dict(self):
#         trunk_qvel_dict =  {
#             'trunk_vel_x': self.trunk_qvel[0],
#             'trunk_vel_y': self.trunk_qvel[1],
#             'trunk_vel_z': self.trunk_qvel[2],
#             'trunk_angvel_x': self.trunk_qvel[3],
#             'trunk_angvel_y': self.trunk_qvel[4],
#             'trunk_angvel_z': self.trunk_qvel[5]}
        
#         return trunk_qvel_dict

#     def create_joints_qvel_dict(self):
#         leg_names = ['FR', 'FL', 'RR', 'RL']
#         joint_names = ['hip_vel', 'thigh_vel', 'calf_vel']
#         joints_qvel_dict = {}
#         for i, leg in enumerate(leg_names):
#             for j, joint in enumerate(joint_names):
#                 key = f"{leg}_{joint}"
#                 joints_qvel_dict[key] = self.joints_qvel[i * 3 + j]

#         return joints_qvel_dict

#     def create_ball_qvel_dict(self):
#         ball_qvel_dict =  {
#             'ball_vel_x': self.ball_qvel[0],
#             'ball_vel_y': self.ball_qvel[1],
#             'ball_vel_z': self.ball_qvel[2],
#             'ball_angvel_x': self.ball_qvel[3],
#             'ball_angvel_y': self.ball_qvel[4],
#             'ball_angvel_z': self.ball_qvel[5]}
        
#         return ball_qvel_dict
    
#     def create_sensed_joints_qpos_dict(self):
#         sensed_joints_qpos_dict = {}

#         position_appendix = "_pos"
#         for joint_name in self.joints:
#             combined_sensed_joint_pos_name = joint_name + position_appendix

#             qpos_sensor_data = self.sensor(combined_sensed_joint_pos_name).data
#             sensed_joints_qpos_dict[combined_sensed_joint_pos_name] = qpos_sensor_data[0]

#         return sensed_joints_qpos_dict

#     def create_sensed_joints_qvel_dict(self):
#         sensed_joints_qvel_dict = {}

#         velocity_appendix = "_vel"
#         for joint_name in self.joints:
#             combined_sensed_joint_vel_name = joint_name + velocity_appendix

#             qvel_sensor_data = self.sensor(combined_sensed_joint_vel_name).data
#             sensed_joints_qvel_dict[combined_sensed_joint_vel_name] = qvel_sensor_data[0]

#         return sensed_joints_qvel_dict

In [ ]:
# env.reset()

# time.sleep(1)

# start_time = time.time()
# max_runtime = 10


# y = move_jointer(4, 5)
# while time.time() - start_time < max_runtime:
#     # x = env.action_space.sample()
#     y = -y
#     env.step(y)
#     statuser(y)


#     # mover()
#     # env.step(x)

#     # print("sensed_joints_qpos_dict: ", env.current_sim_state.sensed_joints_qpos_dict)
#     # print("joints_qpos_dict: ", env.current_sim_state.joints_qpos_dict)


#     # print()
#     # print("sensed_joints_qvel_dict: ", env.current_sim_state.sensed_joints_qvel_dict)
#     # print("joints_qvel_dict: ", env.current_sim_state.joints_qvel_dict)
#     # print("\n \n \n")
    

#     time.sleep(0.1)
#     env.render()